In [1]:

import transformers

In [3]:

transformers.__version__

'4.37.2'

In [4]:
!pip install --upgrade transformers #==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:
import api_keys

In [5]:




def find_json_in_string(string: str) -> str:
    """Finds the JSON object in a string.

    Parameters
    ----------
    string : str
        The string to search for a JSON object.

    Returns
    -------
    json_string : str
    """
    start = string.find("{")
    end = string.rfind("}")
    if start != -1 and end != -1:
        json_string = string[start : end + 1]
    else:
        json_string = "{}"
    return json_string

In [6]:
# from huggingface_hub import login

# api_keys.huggingface

from huggingface_hub import login
login()

#TODO: move to api_keys  


In [8]:
%%time
prompt = """
You are a text classification assistant.
The text to classify:
```
No one cares about me
```
Assign a probability for each possible label: 'loneliness' or 'not loneliness' and return in a JSON format
For instance, return this in JSON format:
'loneliness': <your_score>,
'not loneliness': <your_score>
Do not provide additional text or explanations, just the JSON output.
"""




CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [10]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import LlamaForCausalLM, LlamaTokenizer # llama
import transformers # llama
import torch

# TODO: See how they use it for text classification: (from probs or output layer directly?)
# https://colab.research.google.com/github/bigscience-workshop/petals/blob/main/examples/prompt-tuning-sst2.ipynb

max_length = 500

# model_name = "google/gemma-2b-it"
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
responses = []
if 'gemma' in model_name:
  # Gemma
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
  input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
  outputs = model.generate(**input_ids, max_length = max_length)
  tokenizer.decode(outputs[0])
  # Find the length of the input_ids to know where the original prompt ends
  prompt_length = input_ids["input_ids"].shape[1]
  # Decode only the generated part, skipping the prompt
  generated_text_only = tokenizer.decode(outputs[0][prompt_length:], skip_special_tokens=True)
  responses.append(generated_text_only)
elif 'llama' in model_name:
  # Have to restart session after updating transformers
  from transformers import AutoTokenizer, LlamaForCausalLM

  model = LlamaForCausalLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  prompt = "Hey, are you conscious? Can you talk to me?"
  inputs = tokenizer(prompt, return_tensors="pt")

  # Generate
  generate_ids = model.generate(inputs.input_ids, max_length=max_length)
  response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  responses.append(response)


  # # alternative
  # # Llama
  # pipeline = transformers.pipeline(
  #     "text-generation",
  #     model=model,
  #     torch_dtype=torch.float16,
  #     device_map="auto",
  # )

  # sequences = pipeline(
  #     prompt,
  #     do_sample=True,
  #     top_k=10,
  #     num_return_sequences=1,
  #     eos_token_id=tokenizer.eos_token_id,
  #     max_length=200,
  # )
  # for seq in sequences:
  #     print(f"Result: {seq['generated_text']}")












RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
Failed to import transformers.integrations.aqlm because of the following error (look up to see its traceback):
cannot import name 'is_aqlm_available' from 'transformers.utils' (/usr/local/lib/python3.10/dist-packages/transformers/utils/__init__.py)

In [ ]:
# Have to restart session after updating transformers
from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
%%time


'```\n{\n  "loneliness": 1,\n  "not loneliness": 0\n}'

In [39]:

eval(find_json_in_string(generated_text_only))


{'loneliness': 1, 'not loneliness': 0}

In [33]:

a = '1'

print(a)

In [32]:

print(generated_text_only)

In [28]:
print(generated_text_only)==None

True

In [24]:
tokenizer.decode(outputs[0])


'}'

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
pip install transformers
huggingface-cli login
!pip -i install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.1 MB/s eta 0:00:00


In [1]:
from transformers import pipeline, Conversation
import torch

base_model_name = "bardsai/jaskier-7b-dpo-v5.6"
chatbot = pipeline("conversational", model=base_model_name, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


There is no direct personal connection between the concept of a "bard" and an "ML engineer." A bard is a mythical or literary figure, often a storyteller or musician, while an ML engineer refers to a Machine Learning engineer, a professional in the tech industry. They are unrelated entities, one fictional and the other a real-world occupation.


In [2]:

print(conversation.messages[-1]["content"])

There is no direct personal connection between the concept of a "bard" and an "ML engineer." A bard is a mythical or literary figure, often a storyteller or musician, while an ML engineer refers to a Machine Learning engineer, a professional in the tech industry. They are unrelated entities, one fictional and the other a real-world occupation.


In [4]:
%%time
prompt = """
You are a text classification assistant.
The text to classify:
```
The table is round
```
Assign a probability for each possible label: 'loneliness' or 'not loneliness' and return in a JSON format
For instance, return this in JSON format:
'loneliness': <your_score>,
'not loneliness': <your_score>
Do not provide additional text or explanations, just the JSON output.
"""

conversation = Conversation(prompt)
conversation = chatbot(conversation)
print(conversation.messages[-1]["content"])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINSTINST

In [ ]:

!pip install accelerate



In [ ]:
!pip install --upgrade transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM



In [ ]:
# Setup the environment
!pip install --upgrade huggingface_hub
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00


In [ ]:

!pip install kaggle_secrets

ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
access_token_read = UserSecretsClient().get_secret(api_keys.huggingface)
login(token = access_token_read)

ModuleNotFoundError: No module named 'kaggle_secrets'

In [ ]:


from transformers import AutoTokenizer, AutoModelForCausalLM
# Load the model
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b/2")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b/2")


OSError: Incorrect path_or_model_id: '/kaggle/input/gemma/transformers/2b/2'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# Use the model
input_text = "What is the best thing about Kaggle?"
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

NameError: name 'tokenizer' is not defined